Введение
Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

Реализация в Scikit-Learn
Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом sklearn.feature_extraction.text.TfidfVectorizer. 

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе sklearn.linear_model.Ridge.

Обратите внимание, что признаки LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки LocationNormalized и ContractTime. Он уже реализован в классе sklearn.feature_extraction.DictVectorizer. Пример использования:

In [ ]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, 'nan'). Для этого подходит следующий код:

In [ ]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

**1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).**

In [57]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

In [58]:
train = pd.read_csv('./salary-train.csv')
test = pd.read_csv('./salary-test-mini.csv')

**2. Проведите предобработку:**
+ Приведите тексты к нижнему регистру (text.lower()).

+ Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

+ Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

+ Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.

+ Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

+ Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [32]:
# можно использовать lowercase=True у TfidfVectorizer
train['FullDescription'] = train['FullDescription'].str.lower()
test['FullDescription'] = test['FullDescription'].str.lower()

In [59]:
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [60]:
tf_idf = TfidfVectorizer(min_df=5, lowercase=True)
X_train_description = tf_idf.fit_transform(train['FullDescription'])

In [61]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [62]:
enc = DictVectorizer()
X_train_features = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [63]:
data_new_train = hstack([X_train_features, X_train_description])

In [107]:
X_train_features.shape

(60000, 1766)

In [106]:
len(tf_idf.vocabulary_)

22861

In [110]:
enc.feature_names_

['ContractTime=contract',
 'ContractTime=nan',
 'ContractTime=permanent',
 'LocationNormalized=Abbots Langley',
 'LocationNormalized=Aberaeron',
 'LocationNormalized=Aberdare',
 'LocationNormalized=Aberdeen',
 'LocationNormalized=Aberdeenshire',
 'LocationNormalized=Abergele',
 'LocationNormalized=Abertillery',
 'LocationNormalized=Aberystwyth',
 'LocationNormalized=Abingdon',
 'LocationNormalized=Aboyne',
 'LocationNormalized=Accrington',
 'LocationNormalized=Acton',
 'LocationNormalized=Acton Gate',
 'LocationNormalized=Addlestone',
 'LocationNormalized=Adlington',
 'LocationNormalized=Aintree',
 'LocationNormalized=Airdrie',
 'LocationNormalized=Alcester',
 'LocationNormalized=Aldeburgh',
 'LocationNormalized=Alderley',
 'LocationNormalized=Alderley Edge',
 'LocationNormalized=Aldershot',
 'LocationNormalized=Aldford',
 'LocationNormalized=Aldham Tye',
 'LocationNormalized=Alexandria',
 'LocationNormalized=Alford',
 'LocationNormalized=Alfreton',
 'LocationNormalized=Alfriston',
 'L

In [80]:
data_new_train.shape

(60000, 24627)

**3.  Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.**

In [64]:
clf = Ridge(alpha=1, random_state=241)

In [65]:
clf.fit(X=data_new_train, y=train['SalaryNormalized'])

Ridge(alpha=1, random_state=241)

**4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.**

In [66]:
X_test_description = tf_idf.transform(test['FullDescription'])
X_test_features = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
data_new_test = hstack([X_test_features, X_test_description])

In [67]:
test['SalaryNormalized'] = clf.predict(data_new_test)

In [68]:
test['SalaryNormalized'][0], test['SalaryNormalized'][1]

(56569.22991429722, 37138.92880941285)

In [38]:
with open('./1.txt', 'w') as f:
    f.write("56555.62 37188.32")
f.close()